# **Initial Analysis**

## **Initialise**

In [1]:
!pip install xlrd
import pandas as pd
pd.set_option('display.max_columns', None)

!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import functions_copy1

cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

## **Import Data**

In [2]:
df = pd.read_csv('Missing_Migrants_Global_Figures_allData.csv')

## **Initialise and clean DF**

In [3]:
mdf = functions_copy1.initialise_df_and_add_weather(df)

/Users/madisonwells/Documents/GitHub/data-wrangling-project/functions_copy1.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['latitude', 'longitude']] = df[column_name].str.split(', ', expand=True)
/Users/madisonwells/Documents/GitHub/data-wrangling-project/functions_copy1.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['latitude', 'longitude']] = df[column_name].str.split(', ', expand=True)


In [4]:
mdf = functions_copy1.clean_dataframe(mdf)

In [5]:
mdf

,incident_type,region_of_incident,incident_date,incident_year,month,number_of_dead,minimum_estimated_number_of_missing,total_number_of_dead_and_missing,number_of_survivors,number_of_females,number_of_males,number_of_children,country_of_origin,region_of_origin,cause_of_death,country_of_incident,migration_route,location_of_incident,coordinates,unsd_geographical_grouping,information_source,url,source_quality,latitude,longitude,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,sunrise,sunset,daylight_duration,precipitation_sum,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,id
0,Incident,Mediterranean,2014-01-19,2014,January,12.0,Unknown,12,0.0,9.0,0.0,3.0,"Afghanistan,Syrian Arab Republic",Unknown,Drowning,Greece,Eastern Mediterranean,Waters near Greece while being towed back to T...,"37.2832, 27",Uncategorized,European Council on Refugees and Exiles,http://bit.ly/2aMCwfg,5,37.2832,27,2014-01-19 00:00:00+00:00,3.0,18.302500,14.152500,15.831666,0,0,35914.824219,0.0,18.043148,33.839996,150.063049,1
1,Incident,Mediterranean,2014-02-06,2014,February,15.0,0.0,15,0.0,0.0,1.0,0.0,"Cameroon,Unknown",Unknown,Violence,Spain,Western Mediterranean,Ceuta waters,"35.8793, -5",Uncategorized,L'Espresso,http://bit.ly/U4b8wu,1,35.8793,-5,2014-02-06 00:00:00+00:00,55.0,16.750000,14.600000,15.341666,0,0,38150.320312,2.9,42.737019,60.839996,256.644379,2
2,Incident,Mediterranean,2014-02-17,2014,February,2.0,0.0,2,121.0,0.0,2.0,0.0,Somalia,Eastern Africa,Mixed or unknown,Italy,Central Mediterranean,Italy waters,"34.5645, 16",Uncategorized,Malta Today,http://bit.ly/V7JIqE,1,34.5645,16,2014-02-17 00:00:00+00:00,0.0,17.150000,16.450001,16.862501,0,0,39687.320312,0.0,29.204409,34.919998,159.606415,3
3,Incident,Mediterranean,2014-02-27,2014,February,7.0,Unknown,7,0.0,0.0,0.0,0.0,Somalia,Eastern Africa,Violence,Libya,Central Mediterranean,Libyan Coast,"33.1597, 15",Uncategorized,Libya herald,http://bit.ly/1lvR4Ka,1,33.1597,15,2014-02-27 00:00:00+00:00,53.0,16.600000,15.850000,16.289579,0,0,41075.148438,1.2,27.381308,36.719997,112.557899,4
4,Incident,Mediterranean,2014-03-17,2014,March,1.0,Unknown,1,0.0,0.0,0.0,0.0,Unknown,Unknown,Vehicle accident / death linked to hazardous t...,Libya,Central Mediterranean,Mediterranean,"34.9797, 13",Uncategorized,Global post,http://bit.ly/1lOxCNv,1,34.9797,13,2014-03-17 00:00:00+00:00,2.0,14.850000,14.300000,14.608335,0,0,43263.910156,0.0,32.559605,42.480000,324.796295,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2478,Incident,Mediterranean,2024-03-18,2024,March,2.0,Unknown,2,49.0,0.0,2.0,0.0,Cameroon,Middle Africa,Drowning,Italy,Central Mediterranean,Unspecified location during SAR off the coast ...,"35.44317058, 12.383438",Uncategorized,"IOM Italy, Infomigrants",https://web.archive.org/web/20240322115651/htt...,4,35.44317058,12.383438,2024-03-18 00:00:00+00:00,1.0,17.150000,15.050000,15.733335,0,0,43466.125000,0.0,25.277023,32.039997,216.218918,2479
2479,Incident,Mediterranean,2024-03-19,2024,March,1.0,Unknown,1,0.0,0.0,1.0,0.0,Unknown,Unknown,Mixed or unknown,Spain,Western Mediterranean,Remains found floating off the coast of the Ga...,"35.29638067, -2.931961",Uncategorized,"Melilla Hoy, Nador City, Mahgress",https://web.archive.org/web/20240319035512/htt...,3,35.29638067,-2.931961,2024-03-19 00:00:00+00:00,51.0,27.830000,15.280000,20.269583,0,0,43601.464844,0.1,17.076298,34.200001,121.189079,2480
2480,Incident,Mediterranean,2024-03-22,2024,March,3.0,7.0,10,2.0,0.0,0.0,1.0,Unknown,Northern Africa (P),Drowning,Spain,Western Mediterranean,48 km south of Cape Sacratif in the city of Mo...,"36.009906, -3.344297",Uncategorized,"Spain English, Reuters, SALVAMENTO MARÍTIMO X,...",https://web.archive.org/web/20240328115735/htt...,5,36.009906,-3.344297,2024-03-22 00:00:00+00:00,3.0,18.900000,17.549999,18.079168,0,0,44020.187500,0.0,60.223381,86.039993,72.596481,2481
2481,Inci

## **Checks** 

In [6]:
mdf["region_of_incident"].value_counts()

region_of_incident
Mediterranean    2483
Name: count, dtype: int64

## **Initial Analysis**

#### **Breakdown by incident type**

In [7]:
mdf["incident_type"].value_counts()

incident_type
Incident               2102
Split Incident          368
Cumulative Incident      13
Name: count, dtype: int64

#### **Breakdown by year**

In [8]:
mdf["incident_year"].value_counts().sort_index()

incident_year
2014     74
2015    223
2016    216
2017    236
2018    227
2019    184
2020    229
2021    326
2022    356
2023    344
2024     68
Name: count, dtype: int64

#### **Breakdown by year and route**

In [9]:
incident_breakdown = mdf.groupby(['migration_route', 'incident_year']).size().unstack(fill_value=0)

In [10]:
incident_breakdown

incident_year,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
migration_route,,,,,,,,,,,
Central Mediterranean,57,65,139,163,73,70,127,185,212,238,47
Eastern Mediterranean,12,140,53,11,23,27,23,21,49,39,8
Western Mediterranean,5,18,24,62,131,87,79,120,95,67,13


#### **Breakdown by year and country**

In [69]:
country_year_breakdown = mdf.pivot_table(index='country_of_incident', columns='incident_year', aggfunc='size', fill_value=0)
country_year_breakdown = country_year_breakdown.loc[country_year_breakdown.sum(axis=1).sort_values(ascending=False).index]
country_year_breakdown = country_year_breakdown.iloc[:-5]
country_year_breakdown

incident_year,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
country_of_incident,,,,,,,,,,,
Libya,42,41,109,134,55,52,91,118,131,107,23
Spain,2,14,17,36,37,33,29,50,43,33,10
Tunisia,1,6,7,12,10,10,18,56,42,62,14
Greece,4,99,34,7,4,14,11,11,25,25,2
Algeria,0,0,0,4,37,19,36,45,43,29,1
Morocco,3,4,7,23,61,36,16,27,14,7,4
Italy,10,9,17,16,3,5,9,7,32,64,6
Türkiye,8,42,20,4,15,12,9,10,19,14,3
Egypt,3,7,5,0,0,0,5,0,0,0,0


In [76]:
country_year_breakdown_stack = mdf.groupby(['country_of_incident', 'incident_year']).size()

In [77]:
country_year_breakdown_stack

country_of_incident                                   incident_year
Algeria                                               2017              4
                                                      2018             37
                                                      2019             19
                                                      2020             36
                                                      2021             45
                                                                       ..
Türkiye                                               2021             10
                                                      2022             19
                                                      2023             14
                                                      2024              3
United Kingdom of Great Britain and Northern Ireland  2021              1
Length: 108, dtype: int64

In [78]:
country_year_breakdown_stack.to_csv("country_year_breakdown_stack")

In [71]:
country_year_breakdown.to_csv("country_year_breakdown")

In [ ]:
country_year_breakdown

In [72]:
country_year_long = country_year_breakdown.reset_index().melt(id_vars='country_of_incident', var_name='incident_year', value_name='incident_count')


In [73]:
country_year_long

,country_of_incident,incident_year,incident_count
0,Libya,2014,42
1,Spain,2014,2
2,Tunisia,2014,1
3,Greece,2014,4
4,Algeria,2014,0
...,...,...,...
94,Algeria,2024,1
95,Morocco,2024,4
96,Italy,2024,6
97,Türkiye,2024,3


#### **Breakdown by month and year**

In [11]:
month_order = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
mdf['month'] = pd.Categorical(mdf['month'], categories=month_order, ordered=True)

month_year_breakdown = mdf.groupby(['month', 'incident_year']).size().unstack(fill_value=0)

month_year_breakdown.loc['Total'] = month_year_breakdown.sum()
month_year_breakdown['Total'] = month_year_breakdown.sum(axis=1)

/var/folders/my/jvcytwq53hddwxkgl74_jfz40000gn/T/ipykernel_42969/2081676170.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  month_year_breakdown = mdf.groupby(['month', 'incident_year']).size().unstack(fill_value=0)


In [12]:
month_year_breakdown

incident_year,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Total
month,,,,,,,,,,,,
January,1,5,41,21,17,24,20,27,35,27,23,241
February,3,3,7,12,28,13,7,35,22,28,26,184
March,2,3,18,27,13,18,5,22,17,23,19,167
April,3,11,11,22,17,11,5,27,16,52,0,175
May,6,9,17,27,10,18,5,31,25,24,0,172
June,11,5,11,44,28,13,9,20,35,34,0,210
July,16,11,13,14,16,10,21,44,33,24,0,202
August,7,29,15,8,14,14,26,35,24,45,0,217
September,6,33,13,9,26,24,42,17,48,19,0,237


In [13]:
month_year_fatalities = mdf.groupby(['month', 'incident_year'])['total_number_of_dead_and_missing'].sum().unstack(fill_value=0)
month_year_fatalities.loc['Total'] = month_year_fatalities.sum()
month_year_fatalities['Total'] = month_year_fatalities.sum(axis=1)

/var/folders/my/jvcytwq53hddwxkgl74_jfz40000gn/T/ipykernel_42969/2703847917.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  month_year_fatalities = mdf.groupby(['month', 'incident_year'])['total_number_of_dead_and_missing'].sum().unstack(fill_value=0)


In [14]:
month_year_fatalities

incident_year,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Total
month,,,,,,,,,,,,
January,12,82,370,254,243,216,123,118,227,76,117,1838
February,24,345,55,231,197,26,119,146,120,247,153,1663
March,10,61,324,318,67,120,35,148,145,241,189,1658
April,50,1494,630,313,110,64,19,223,293,562,0,3758
May,329,96,1178,623,48,108,12,200,178,146,0,2918
June,318,33,389,539,629,81,99,113,145,781,0,3127
July,864,230,226,132,224,306,87,311,164,99,0,2643
August,616,689,62,151,134,173,141,133,167,347,0,2613
September,813,269,368,115,241,137,240,41,402,83,0,2709


#### **Breakdown by year and total number of people dead and missing**

In [15]:
year_fatalities_pivot = pd.pivot_table(mdf, values='total_number_of_dead_and_missing', index='incident_year', aggfunc='sum')
year_fatalities_pivot.loc['Total'] = year_fatalities_pivot.sum()
year_fatalities_pivot

,total_number_of_dead_and_missing
incident_year,
2014,3289
2015,4055
2016,5136
2017,3139
2018,2337
2019,1885
2020,1449
2021,2048
2022,2411


#### **Breakdown by month and total number of people dead and missing**

In [16]:
month_year_fatalities_pivot = pd.pivot_table(mdf, values='total_number_of_dead_and_missing', index='month', columns='incident_year', aggfunc='sum', fill_value=0)
month_year_fatalities_pivot.loc['Total'] = month_year_fatalities_pivot.sum()
month_year_fatalities_pivot['Total'] = month_year_fatalities_pivot.sum(axis=1)
month_year_fatalities_pivot

incident_year,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Total
month,,,,,,,,,,,,
January,12,82,370,254,243,216,123,118,227,76,117,1838
February,24,345,55,231,197,26,119,146,120,247,153,1663
March,10,61,324,318,67,120,35,148,145,241,189,1658
April,50,1494,630,313,110,64,19,223,293,562,0,3758
May,329,96,1178,623,48,108,12,200,178,146,0,2918
June,318,33,389,539,629,81,99,113,145,781,0,3127
July,864,230,226,132,224,306,87,311,164,99,0,2643
August,616,689,62,151,134,173,141,133,167,347,0,2613
September,813,269,368,115,241,137,240,41,402,83,0,2709


In [17]:
mdf

,incident_type,region_of_incident,incident_date,incident_year,month,number_of_dead,minimum_estimated_number_of_missing,total_number_of_dead_and_missing,number_of_survivors,number_of_females,number_of_males,number_of_children,country_of_origin,region_of_origin,cause_of_death,country_of_incident,migration_route,location_of_incident,coordinates,unsd_geographical_grouping,information_source,url,source_quality,latitude,longitude,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,sunrise,sunset,daylight_duration,precipitation_sum,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,id
0,Incident,Mediterranean,2014-01-19,2014,January,12.0,Unknown,12,0.0,9.0,0.0,3.0,"Afghanistan,Syrian Arab Republic",Unknown,Drowning,Greece,Eastern Mediterranean,Waters near Greece while being towed back to T...,"37.2832, 27",Uncategorized,European Council on Refugees and Exiles,http://bit.ly/2aMCwfg,5,37.2832,27,2014-01-19 00:00:00+00:00,3.0,18.302500,14.152500,15.831666,0,0,35914.824219,0.0,18.043148,33.839996,150.063049,1
1,Incident,Mediterranean,2014-02-06,2014,February,15.0,0.0,15,0.0,0.0,1.0,0.0,"Cameroon,Unknown",Unknown,Violence,Spain,Western Mediterranean,Ceuta waters,"35.8793, -5",Uncategorized,L'Espresso,http://bit.ly/U4b8wu,1,35.8793,-5,2014-02-06 00:00:00+00:00,55.0,16.750000,14.600000,15.341666,0,0,38150.320312,2.9,42.737019,60.839996,256.644379,2
2,Incident,Mediterranean,2014-02-17,2014,February,2.0,0.0,2,121.0,0.0,2.0,0.0,Somalia,Eastern Africa,Mixed or unknown,Italy,Central Mediterranean,Italy waters,"34.5645, 16",Uncategorized,Malta Today,http://bit.ly/V7JIqE,1,34.5645,16,2014-02-17 00:00:00+00:00,0.0,17.150000,16.450001,16.862501,0,0,39687.320312,0.0,29.204409,34.919998,159.606415,3
3,Incident,Mediterranean,2014-02-27,2014,February,7.0,Unknown,7,0.0,0.0,0.0,0.0,Somalia,Eastern Africa,Violence,Libya,Central Mediterranean,Libyan Coast,"33.1597, 15",Uncategorized,Libya herald,http://bit.ly/1lvR4Ka,1,33.1597,15,2014-02-27 00:00:00+00:00,53.0,16.600000,15.850000,16.289579,0,0,41075.148438,1.2,27.381308,36.719997,112.557899,4
4,Incident,Mediterranean,2014-03-17,2014,March,1.0,Unknown,1,0.0,0.0,0.0,0.0,Unknown,Unknown,Vehicle accident / death linked to hazardous t...,Libya,Central Mediterranean,Mediterranean,"34.9797, 13",Uncategorized,Global post,http://bit.ly/1lOxCNv,1,34.9797,13,2014-03-17 00:00:00+00:00,2.0,14.850000,14.300000,14.608335,0,0,43263.910156,0.0,32.559605,42.480000,324.796295,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2478,Incident,Mediterranean,2024-03-18,2024,March,2.0,Unknown,2,49.0,0.0,2.0,0.0,Cameroon,Middle Africa,Drowning,Italy,Central Mediterranean,Unspecified location during SAR off the coast ...,"35.44317058, 12.383438",Uncategorized,"IOM Italy, Infomigrants",https://web.archive.org/web/20240322115651/htt...,4,35.44317058,12.383438,2024-03-18 00:00:00+00:00,1.0,17.150000,15.050000,15.733335,0,0,43466.125000,0.0,25.277023,32.039997,216.218918,2479
2479,Incident,Mediterranean,2024-03-19,2024,March,1.0,Unknown,1,0.0,0.0,1.0,0.0,Unknown,Unknown,Mixed or unknown,Spain,Western Mediterranean,Remains found floating off the coast of the Ga...,"35.29638067, -2.931961",Uncategorized,"Melilla Hoy, Nador City, Mahgress",https://web.archive.org/web/20240319035512/htt...,3,35.29638067,-2.931961,2024-03-19 00:00:00+00:00,51.0,27.830000,15.280000,20.269583,0,0,43601.464844,0.1,17.076298,34.200001,121.189079,2480
2480,Incident,Mediterranean,2024-03-22,2024,March,3.0,7.0,10,2.0,0.0,0.0,1.0,Unknown,Northern Africa (P),Drowning,Spain,Western Mediterranean,48 km south of Cape Sacratif in the city of Mo...,"36.009906, -3.344297",Uncategorized,"Spain English, Reuters, SALVAMENTO MARÍTIMO X,...",https://web.archive.org/web/20240328115735/htt...,5,36.009906,-3.344297,2024-03-22 00:00:00+00:00,3.0,18.900000,17.549999,18.079168,0,0,44020.187500,0.0,60.223381,86.039993,72.596481,2481
2481,Inci

#### **Breakdown by gender**

In [18]:
mdf['number_of_dead'] = pd.to_numeric(mdf['number_of_dead'], errors='coerce')
mdf['minimum_estimated_number_of_missing'] = pd.to_numeric(mdf['minimum_estimated_number_of_missing'], errors='coerce')
mdf['total_number_of_dead_and_missing'] = pd.to_numeric(mdf['total_number_of_dead_and_missing'], errors='coerce')

mdf_known_gender = mdf[mdf['number_of_females'] != 'unknown']
mdf_unknown_gender = mdf[mdf['number_of_females'] == 'unknown']

grouped_known_gender = mdf_known_gender.groupby(['incident_year']).agg({
    'number_of_females': 'sum',
    'number_of_males': 'sum',
    'total_number_of_dead_and_missing': 'sum'
}).reset_index()

grouped_known_gender['unknown'] = grouped_known_gender['total_number_of_dead_and_missing'] - grouped_known_gender['number_of_females'] - grouped_known_gender['number_of_males']

grouped_known_gender.rename(columns={'incident_year': 'Year'}, inplace=True)

grouped_known_gender = grouped_known_gender[['Year', 'number_of_females', 'number_of_males', 'unknown', 'total_number_of_dead_and_missing']]

In [19]:
grouped_known_gender

,Year,number_of_females,number_of_males,unknown,total_number_of_dead_and_missing
0,2014,37.0,105.0,3147.0,3289
1,2015,168.0,415.0,3472.0,4055
2,2016,266.0,496.0,4374.0,5136
3,2017,236.0,480.0,2423.0,3139
4,2018,179.0,509.0,1649.0,2337
5,2019,170.0,220.0,1495.0,1885
6,2020,115.0,434.0,900.0,1449
7,2021,166.0,748.0,1134.0,2048
8,2022,136.0,260.0,2015.0,2411
9,2023,100.0,244.0,2761.0,3105


#### **Breakdown by age (children/adults)**

In [31]:
mdf['number_of_children'] = pd.to_numeric(mdf['number_of_children'])
children_breakdown = mdf.groupby('incident_year')['number_of_children'].sum()
children_breakdown_df = children_breakdown.reset_index(name='total_children')
children_breakdown_df

,incident_year,total_children
0,2014,27.0
1,2015,277.0
2,2016,179.0
3,2017,77.0
4,2018,125.0
5,2019,81.0
6,2020,81.0
7,2021,81.0
8,2022,107.0
9,2023,161.0


#### **Breakdown by source**

In [32]:
mdf["information_source"].value_counts()

information_source
IOM Libya                           394
IOM Italy                            78
UNHCR                                75
CIPIMD                               60
IOM Tunisia                          32
                                   ... 
Ceuta al día, El Pueblo de Ceuta      1
El Pueblo de Ceuta                    1
Europa Sur, ABC                       1
Alarm Phone, Sea-Eye                  1
MARE*GO, ANSA                         1
Name: count, Length: 1138, dtype: int64

#### **Breakdown by cause of death**

In [33]:
mdf["cause_of_death"].value_counts()

cause_of_death
Drowning                                                                  2075
Mixed or unknown                                                           223
Harsh environmental conditions / lack of adequate shelter, food, water      70
Vehicle accident / death linked to hazardous transport                      50
Sickness / lack of access to adequate healthcare                            30
Violence                                                                    23
Accidental death                                                            12
Name: count, dtype: int64

#### **Breakdown by county of incident**

In [34]:
mdf["country_of_incident"].value_counts()[:10]

country_of_incident
Libya      903
Spain      304
Tunisia    238
Greece     236
Algeria    214
Morocco    202
Italy      178
Türkiye    156
Egypt       20
Malta       14
Name: count, dtype: int64

#### **Summary statistics for weather**

In [35]:
summary_stats = mdf[['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'wind_gusts_10m_max']].describe()
summary_stats.round(2)

,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_gusts_10m_max
count,2483.00,2483.00,2483.00,2483.00
mean,19.93,1.27,23.69,38.78
std,5.51,5.03,9.43,14.25
min,1.62,0.00,5.69,10.44
25%,15.53,0.00,16.92,28.44
50%,19.55,0.00,21.89,36.00
75%,24.51,0.00,28.98,46.08
max,35.10,71.20,71.32,104.04


## **Weather Analysis**

#### **Weather code frequency**

In [79]:
weather_codes = mdf["weather_code"].value_counts()

In [80]:
weather_codes

weather_code
1.0     738
0.0     670
51.0    289
3.0     278
2.0     197
63.0     94
53.0     93
61.0     79
55.0     31
65.0      9
73.0      2
71.0      2
75.0      1
Name: count, dtype: int64

In [81]:
weather_codes.to_csv("weather_codes")

0	Clear sky
1, 2, 3	Mainly clear, partly cloudy, and overcast
45, 48	Fog and depositing rime fog
51, 53, 55	Drizzle: Light, moderate, and dense intensity
56, 57	Freezing Drizzle: Light and dense intensity
61, 63, 65	Rain: Slight, moderate and heavy intensity
66, 67	Freezing Rain: Light and heavy intensity
71, 73, 75	Snow fall: Slight, moderate, and heavy intensity
77	Snow grains
80, 81, 82	Rain showers: Slight, moderate, and violent
85, 86	Snow showers slight and heavy
95 *	Thunderstorm: Slight or moderate
96, 99 *	Thunderstorm with slight and heavy hail

#### **Wind Analysis**

In [37]:
beaufort_scale = {
    "Calm": (0, 1),
    "Light air": (2, 5),
    "Light breeze": (6, 11),
    "Gentle breeze": (12, 19),
    "Moderate breeze": (20, 28),
    "Fresh breeze": (29, 38),
    "Strong breeze": (39, 49),
    "Moderate gale": (50, 61),
    "Fresh gale": (62, 74),
    "Strong gale": (75, 88),
    "Whole gale": (89, 102),
    "Storm": (103, 117),
    "Violent storm": (118, 133),
    "Hurricane force": (134, float("inf"))  
}

def map_to_beaufort(wind_speed_kmh):
    for category, (lower, upper) in beaufort_scale.items():
        if lower <= wind_speed_kmh <= upper:
            return category

mdf["beaufort_scale_category"] = mdf["wind_speed_10m_max"].apply(lambda x: map_to_beaufort(x))

wind_speed_counts = mdf["beaufort_scale_category"].value_counts().round(2)
wind_speed_counts

beaufort_scale_category
Moderate breeze    801
Gentle breeze      692
Fresh breeze       425
Strong breeze      145
Light breeze       125
Moderate gale       24
Fresh gale           5
Name: count, dtype: int64

In [38]:
wind_speed_counts.to_csv("wind_speed_counts")

#### **Breakdown by mean temperature**

In [39]:
temperature_categories = {
    "Freezing": (-float("inf"), 0),
    "Cold": (0, 10),
    "Cool": (10, 20),
    "Mild": (20, 25),
    "Warm": (25, 30),
    "Hot": (30, float("inf"))
}

def map_to_temperature_category(temperature):
    for category, (lower, upper) in temperature_categories.items():
        if lower <= temperature <= upper:
            return category
    return "Unknown"

mdf["temperature_category"] = mdf["temperature_2m_mean"].apply(lambda x: map_to_temperature_category(x))

temperature_counts = mdf["temperature_category"].value_counts().round(2)
temperature_counts

temperature_category
Cool    1242
Mild     623
Warm     515
Cold      59
Hot       44
Name: count, dtype: int64

In [40]:
temperature_counts.to_csv("temperature_counts")

# **Secondry Data**

In [84]:
file_path = 'mediterranean-data(2024-04-11).xlsx'
df = pd.read_excel(file_path)

In [102]:
pushbacks_df = df[df["field_details"].isin(["Interceptions by Turkish Coast Guard", "Interceptions by Tunisian Coast Guard", "Interceptions by Libyan Coast Guard"])]
pushbacks_df_excluded = pushbacks_df[pushbacks_df["Year"] != 2024]
pushbacks_sum_per_year_excluded = pushbacks_df_excluded.groupby(["field_details", "Year"])["field_count"].sum().reset_index(name="total_field_count")
pushbacks_pivot_table = pushbacks_sum_per_year_excluded.pivot_table(index='field_details', columns='Year', values='total_field_count', aggfunc='sum')
pushbacks_pivot_table_rounded = pushbacks_pivot_table.astype(int)

In [103]:
pushbacks_pivot_table_rounded

Year,2016,2017,2018,2019,2020,2021,2022,2023
field_details,,,,,,,,
Interceptions by Libyan Coast Guard,14332,18900,15428,1428,11891,32425,24684,17025
Interceptions by Tunisian Coast Guard,1105,3130,4091,1028,13466,25657,33372,42370
Interceptions by Turkish Coast Guard,37130,19084,25398,60543,20380,23676,49522,57226


In [114]:
pushbacks_long_format = pushbacks_pivot_table_rounded.reset_index().melt(id_vars='field_details', var_name='Year', value_name='total_field_count')

In [118]:
pushbacks_long_format.to_csv("pushbacks_long")

In [116]:
pushbacks_pivot_table_rounded.to_csv("pushbacks_pivot_table")

In [106]:
arrivals_df = df[df["field_activity"] == "Irregular arrivals in Europe"]
total_field_count = arrivals_df["field_count"].sum()
grouped_data = arrivals_df.groupby(['Year', 'field_details'])['field_count'].sum().reset_index()
filtered_grouped_data = grouped_data[~grouped_data['field_details'].isin(["Arrivals in Cyprus", "Land arrivals in Spain"])]
filtered_grouped_data = filtered_grouped_data[filtered_grouped_data["Year"] != 2024]
arrivals_pivot_table = filtered_grouped_data
arrivals_pivot_table = filtered_grouped_data.pivot_table(index='field_details', columns='Year', values='field_count', aggfunc='sum')
arrivals_pivot_table_rounded = arrivals_pivot_table.astype(int)

In [107]:
arrivals_pivot_table_rounded

Year,2016,2017,2018,2019,2020,2021,2022,2023
field_details,,,,,,,,
Sea arrivals in Greece,173614,29501,32742,62445,9290,4184,12395,39778
Sea arrivals in Italy,181436,119370,23370,11471,34157,67490,105134,157652
Sea arrivals in Malta,0,0,1445,3405,2281,838,440,381
Sea arrivals in Spain,7490,21546,56191,23416,17083,17784,13248,15708


In [111]:
arrivals_long_format = arrivals_pivot_table_rounded.reset_index().melt(id_vars='field_details', var_name='Year', value_name='field_count')

In [112]:
arrivals_long_format.to_csv("arrivals_long")